In [19]:
import os
import json
import boto3

import pandas as pd

from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

ACCESS_KEY = os.getenv("ID_KEY_ACCESS")
ACCESS_KEY_SECRETE = os.getenv("KEY_SECRET")
BUCKET_NAME = os.getenv("BUCKETNAME")

In [20]:
df_1 = pd.read_json("../hotel_1.json")
df_2 = pd.read_json("../hotel_2.json")
df_3 = pd.read_json("../temps.json")

In [21]:
# merge des 3 dataframe
def merge_outer(data_1:pd.DataFrame, data_2:pd.DataFrame, on:str) -> pd.DataFrame:
    df = pd.merge(data_1, data_2, how='outer', on=on)
    return df

df = merge_outer(df_1, df_2, "hotel")
df = merge_outer(df, df_3, "id")

In [22]:
# drop des hotel ou il manque la note
df.dropna()

,id,ville,hotel,url,note,description,latitutde,longitude,jour_1,jour_2,jour_3,jour_4,jour_5,lat,long
0,1,Le Mont-Saint-Michel,Hôtel Vert,https://www.booking.com/hotel/fr/vert.fr.html?...,"8,0",Vous pouvez bénéficier d'une réduction Genius ...,48.614700,-1.509617,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
1,1,Le Mont-Saint-Michel,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,2",Installé dans des espaces verts à seulement 2 ...,48.614247,-1.510545,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
2,1,Le Mont-Saint-Michel,La Mère Poulard,https://www.booking.com/hotel/fr/la-mere-poula...,"7,0",Vous pouvez bénéficier d'une réduction Genius ...,48.635085,-1.510540,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
3,1,Le Mont-Saint-Michel,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,1",L'hôtel De La Digue est un établissement tradi...,48.616882,-1.510918,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
4,1,Le Mont-Saint-Michel,Les Terrasses Poulard,https://www.booking.com/hotel/fr/les-terrasses...,"7,3",Vous pouvez bénéficier d'une réduction Genius ...,48.635349,-1.510379,"[6.33, nuageux]","[7.44, légère pluie]","[5.41, peu nuageux]","[4.74, ciel dégagé]","[4.21, ciel dégagé]",48.635523,-1.510257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
870,35,"1er arr., Lyon",Hamac Suites - Le Plâtre - Place des Terreaux,https://www.booking.com/hotel/fr/hamac-suites-...,"8,7","Doté d'une connexion Wi-Fi gratuite, le Hamac ...",45.766106,4.833282,"[7.62, nuageux]","[6.94, peu nuageux]","[5.69, nuageux]","[5.16, couvert]","[3.73, ciel dégagé]",46.159113,-1.152043
871,35,"7e arr., Lyon",B&B HOTEL LYON Centre Perrache Berthelot,https://www.booking.com/hotel/fr/b-amp-b-lyon-...,"7,8","Ouvert en décembre 2017, le B&B HOTEL LYON Cen...",45.747809,4.835540,"[7.62, nuageux]","[6.94, peu nuageux]","[5.69, nuageux]","[5.16, couvert]","[3.73, ciel dégagé]",46.159113,-1.152043
872,35,"2e arr., Lyon",Hôtel Charlemagne,https://www.booking.com/hotel/fr/hotelbestwest...,"7,8",Vous pouvez bénéficier d'une réduction Genius ...,45.746023,4.823894,"[7.62, nuageux]","[6.94, peu nuageux]","[5.69, nuageux]","[5.16, couvert]","[3.73, ciel dégagé]",46.159113,-1.152043
873,35,"5e arr., Lyon",MiHotel Tour Rose,https://www.booking.com/hotel/fr/mihotel-tour-...,"8,4",Vous pouvez bénéficier d'une réduction Genius ...,45.762725,4.826820,"[7.62, nuageux]","[6.94, peu nuageux]","[5.69, nuageux]","[5.16, couvert]","[3.73, ciel dégagé]",46.159113,-1.152043


In [23]:
# sauvegarde du dataframe en csv pour envoie dans un s3
csv = df.to_csv()
df.to_csv("hotel.csv")

In [2]:
# création d'une session aws
session = boto3.Session(aws_access_key_id=ACCESS_KEY, 
                        aws_secret_access_key=ACCESS_KEY_SECRETE)
s3 = session.resource("s3")

In [ ]:
# création d'un bucket s3
bucket = s3.create_bucket(Bucket=BUCKET_NAME, CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [9]:
# ajout du fichier csv dans le compartiment bucket s3
put_object = bucket.put_object(Key="hotel.csv", Body=csv)

In [ ]:
# recuperation du fichier csv dans un s3
df = pd.read_csv(f's3://{BUCKET_NAME}/hotel.csv')

In [24]:
df_temps = df[["id", "lat", "long", "jour_1", "jour_2" ,"jour_3", "jour_4", "jour_5"]]

In [28]:
df_temps["jour_1"] = df["jour_1"].apply(lambda x : x[0])
df_temps["jour_2"] = df["jour_2"].apply(lambda x : x[0])
df_temps["jour_3"] = df["jour_3"].apply(lambda x : x[0])
df_temps["jour_4"] = df["jour_4"].apply(lambda x : x[0])
df_temps["jour_5"] = df["jour_5"].apply(lambda x : x[0])

C:\Users\jerem\AppData\Local\Temp\ipykernel_5244\2503917258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps["jour_1"] = df["jour_1"].apply(lambda x : x[0])
C:\Users\jerem\AppData\Local\Temp\ipykernel_5244\2503917258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temps["jour_2"] = df["jour_2"].apply(lambda x : x[0])
C:\Users\jerem\AppData\Local\Temp\ipykernel_5244\2503917258.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

In [29]:
df_temps

,id,lat,long,jour_1,jour_2,jour_3,jour_4,jour_5
0,1,48.635523,-1.510257,6.33,7.44,5.41,4.74,4.21
1,1,48.635523,-1.510257,6.33,7.44,5.41,4.74,4.21
2,1,48.635523,-1.510257,6.33,7.44,5.41,4.74,4.21
3,1,48.635523,-1.510257,6.33,7.44,5.41,4.74,4.21
4,1,48.635523,-1.510257,6.33,7.44,5.41,4.74,4.21
...,...,...,...,...,...,...,...,...
870,35,46.159113,-1.152043,7.62,6.94,5.69,5.16,3.73
871,35,46.159113,-1.152043,7.62,6.94,5.69,5.16,3.73
872,35,46.159113,-1.152043,7.62,6.94,5.69,5.16,3.73
873,35,46.159113,-1.152043,7.62,6.94,5.69,5.16,3.73


In [17]:
# engine = create_engine(f"postgresql+psycopg2://{YOUR_USERNAME}:{YOUR_PASSWORD}@{YOUR_HOSTNAME}/postgres", echo=True)
engine = create_engine('sqlite:///:memory:', echo=True)

In [18]:
df.to_sql("hotel", engine)

2022-12-04 12:10:43,088 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hotel")
2022-12-04 12:10:43,088 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-04 12:10:43,089 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hotel")
2022-12-04 12:10:43,089 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-12-04 12:10:43,091 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-12-04 12:10:43,092 INFO sqlalchemy.engine.Engine 
CREATE TABLE hotel (
	"index" BIGINT, 
	"Unnamed: 0" BIGINT, 
	ville_rechercher TEXT, 
	"ville_précise" TEXT, 
	hotel TEXT, 
	url TEXT, 
	note TEXT, 
	description TEXT, 
	latitutde FLOAT, 
	longitude FLOAT, 
	"2022-12-04 15:00:00" TEXT, 
	"2022-12-05 15:00:00" TEXT, 
	"2022-12-06 15:00:00" TEXT, 
	"2022-12-07 15:00:00" TEXT, 
	"2022-12-08 15:00:00" TEXT
)


2022-12-04 12:10:43,092 INFO sqlalchemy.engine.Engine [no key 0.00038s] ()
2022-12-04 12:10:43,093 INFO sqlalchemy.engine.Engine CREATE INDEX ix_hotel_index ON hotel ("index")
2022-12-04 12:10:43,093 INFO sql

878

In [19]:
conn = engine.connect()

In [21]:
statement = text("SELECT DISTINCT ville_précise FROM hotel")

In [22]:
result = conn.execute(statement)

2022-12-04 12:12:18,073 INFO sqlalchemy.engine.Engine SELECT DISTINCT ville_précise FROM hotel
2022-12-04 12:12:18,073 INFO sqlalchemy.engine.Engine [generated in 0.00067s] ()


In [23]:
result.fetchall()

[('Le Mont-Saint-Michel',),
 ('Ardevon',),
 ('Pontorson',),
 ('Beauvoir',),
 ('Saint-Malo',),
 ('Paramé, Saint-Malo',),
 ('Intra muros, Saint-Malo',),
 ('Saint-Servan, Saint-Malo',),
 ('Sillon, Saint-Malo',),
 ('Bayeux',),
 ('Centre-ville de Rouen, Rouen',),
 ('Rouen',),
 ('Perret, Le Havre',),
 ('Le Havre',),
 ('6e arr., Paris',),
 ('13e arr., Paris',),
 ('9e arr., Paris',),
 ('5e arr., Paris',),
 ('10e arr., Paris',),
 ('17e arr., Paris',),
 ('14e arr., Paris',),
 ('16e arr., Paris',),
 ('3e arr., Paris',),
 ('15e arr., Paris',),
 ('11e arr., Paris',),
 ('8e arr., Paris',),
 ('19e arr., Paris',),
 ('2e arr., Paris',),
 ('12e arr., Paris',),
 ('Amiens',),
 ('Centre de Lille, Lille',),
 ('Lille Sud, Lille',),
 ('Vieux-Lille, Lille',),
 ('Vauban, Lille',),
 ('Lille',),
 ('Faubourg de Béthune, Lille',),
 ('Wazemmes, Lille',),
 ('Fives, Lille',),
 ('Centre de Strasbourg - Petite France - Cathédrale, Strasbourg',),
 ('Strasbourg',),
 ('Quartier Européen - Orangerie, Strasbourg',),
 ('Neudo